In [ ]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from config import *

# Create a new instance of the Chrome driver
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get(target_url)

xpath = "/html/body/form/div[3]/main/div[1]/div/div/div/div[1]/div[1]/main/div/div[1]/div/div/div[2]/div[2]/div[1]/div[1]/span/div/button"
# path to specific case. Replace with xpath of other case as needed.

button = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH, xpath))
    )
button.click()

pager = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.NAME, "pageSelector"))
)

page_selector = Select(pager)
options = page_selector.options
links = []
for option in options:
    option.click()
    WebDriverWait(driver,10).until( # wait for links to load
        EC.presence_of_element_located((By.XPATH, "/html/body/form/div[3]/main/div[1]/div/div/div/div[1]/div[1]/main/div/div[2]/div/div[3]/div[2]/div[2]/div/div[1]/a"))
    )
    for i in range(20): # clunky iteration because elements don't have neat selectors.
        try: # so that it doesn't break on the last page
            element = driver.find_element_by_xpath("/html/body/form/div[3]/main/div[1]/div/div/div/div[1]/div[1]/main/div/div[2]/div/div[3]/div[2]/div[{}]".format(i+2))
            text = element.get_attribute("innerText") # gets the name of the filing, date, AE number, and presence of attachment.
            parts = text.split('\n')

            html = element.get_attribute("outerHTML")
            link = target_url[:-15] + html[html.find('href')+6:(html.find('.pdf')+4)] # sinful but it works.
            
            parts.append(link)
            links.append(parts)
        except:
            print("element not found, possibly last page")

df = pd.DataFrame(links, columns=['FilingName', 'FilingDate', 'AENumber', 'Attachment', 'Hyperlink'])
df.to_csv('linktable.csv')

In [8]:
driver.quit()

In [7]:
import pymongo
import pandas
import config 

client = pymongo.MongoClient(config.mongodb_cs)
db = client.Transcripts
testdata = db.linktable
records_df = pandas.read_csv('linktable.csv', header=0, index_col=0,parse_dates=['FilingDate'])
records_dict = records_df.to_dict(orient='records')

db.drop_collection(testdata)
testdata.insert_many(records_dict)